# Compute Statuscodes
1. check where each train is at the moment
2. collect trip ids of trains that are somewhere on the map right now
3. check between which stations each trip is
4. find the track each trip is on
5. calculate the position of each train on their respective track
6. lookup the track segment each track is on
7. put the statuscode together and output

In [27]:
import datetime
from dotenv import load_dotenv
from os import getenv
import pandas as pd

load_dotenv()

STOPTIMES_PATH = 'stoptimes.csv'

# chatgpt generiert lol
import datetime
def dateToDBDate(date: datetime.date) -> str:
    """
    Wandelt ein datetime.date Objekt in einen DB-Date-String "YYMMDD" um.
    """
    return date.strftime("%y%m%d")


def datetimeToDBDatetime(dt: datetime.datetime) -> str:
    """
    Wandelt ein datetime.datetime Objekt in einen DB-Datetime-String "YYMMDDHHMM" um.
    """
    return dt.strftime("%y%m%d%H%M")


def DBDatetimeToDatetime(dbDate: str) -> datetime.datetime:
    """
    Wandelt einen DB-Datetime-String "YYMMDDHHMM" in ein datetime.datetime Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d%H%M")


def DBDateToDate(dbDate: str) -> datetime.date:
    """
    Wandelt einen DB-Date-String "YYMMDD" in ein datetime.date Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d").date()

def datetimeToDBDateAndHourTuple(dt: datetime.datetime):
    date = dt.strftime("%y%m%d")
    hour = dt.strftime("%H")
    return (date, hour)
    
       
print(dateToDBDate(datetime.date(2025, 8, 10)))
print(datetimeToDBDatetime(datetime.datetime(2025, 8, 10, 12, 22)))
print(DBDateToDate("250810"))
print(DBDatetimeToDatetime("2508101222"))
print(datetimeToDBDateAndHourTuple(datetime.datetime(2025, 8, 10, 12, 22)))

250810
2508101222
2025-08-10
2025-08-10 12:22:00
('250810', '12')


In [28]:
df_stoptimes = pd.read_csv(STOPTIMES_PATH, dtype=str)


# find active trip ids

current_dbdatetime = datetimeToDBDatetime(datetime.datetime.now())
    
# train is departed, when the departure time was before now
df_stoptimes['has_departed_station'] = df_stoptimes['departure_dbdatetime'] < current_dbdatetime

# find tripids with one stop time and remove them, as for identifying a position on a track at least two stops are needed.
tripids_with_one_stoptime_mask = df_stoptimes.groupby("trip_id")["trip_id"].transform("count") == 1
df_stoptimes = df_stoptimes.drop(df_stoptimes[tripids_with_one_stoptime_mask].index)

# find tripids where all stops have been departed and remove them, as that means the trip has ended
tripids_ended_mask = df_stoptimes.groupby("trip_id")["has_departed_station"].transform("all")
df_stoptimes = df_stoptimes.drop(df_stoptimes[tripids_ended_mask].index)

# find tripids where not any stops have been departed and remove them, as that means the trip has not started yet
tripids_not_started_mask = ~df_stoptimes.groupby("trip_id")["has_departed_station"].transform("any")
df_stoptimes = df_stoptimes.drop(df_stoptimes[tripids_not_started_mask].index)

# only the active trips remain

no_active_trips = df_stoptimes['trip_id'].unique().shape[0]
print(f"found {no_active_trips} active trips")

df_stoptimes.to_csv('temp.csv', index=False)

found 9 active trips


In [29]:
# find out, between which two stations a train is traveling

# a train traveling from A to B and a train coming from A and standing at B are treated the same. This is because the station and its 5 or 10km radius are displayed by the same LED so it makes no difference. A train enters a segment the moment it departs at the starting station and exits the moment it departs at the ending station.
# therefore the train is between stations A and B, with station A being the last station it departed and station B being the first station it has not departed from a time perspective. 